In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
# Nome dos atributos
columnsNames = ['Outlook','Temperature','Humidity','Wind','PlayTennis']

# carregando a base de dados a ser usada
base = pd.read_csv("exemplo_IA.csv", header=None, names=columnsNames)